## Start

Code réalisé par Léo Demelle, Jules Sayad-Barth et Jiaming Miao

### Imports

In [1]:
import numpy as np
import strawberryfields as sf
from strawberryfields import ops

### Constants

In [2]:
cutoff = 4
angle45 = np.pi/4

# kets 1-mode
ket0 = np.array([1,0,0,0], dtype=complex)
ket1 = np.array([0,1,0,0], dtype=complex)

### Utilitary functions

In [3]:
def tensor_ket(states):
    out = states[0]
    for s in states[1:]:
        out = np.kron(out, s)
    return out

#
---

## Figure 1: $NS_{-1}$ gate (probabilité 1/4)

In [4]:
def NS_1(q) :
    """
    list -> None
    Input: should be a list of the target mode, the first ancilla mode (state 1) and the second ancilla mode (empty/state 0)
    Compute the NSx gate for x == -1
    """

    theta1, theta2, theta3 = np.deg2rad(22.5), np.deg2rad(65.5302), np.deg2rad(-22.5)
    phi1, phi2, phi3, phi4 = np.deg2rad(0.0), np.deg2rad(0.0), np.deg2rad(0.0), np.deg2rad(180.0)

    ops.Rgate(phi4)          | q[0]
    ops.BSgate(theta1,phi1)  | (q[1], q[2])
    ops.BSgate(theta2,phi2)  | (q[0], q[1])
    ops.BSgate(theta3,phi3)  | (q[1], q[2])

In [5]:
alpha, beta, gamma = np.sqrt(1/3), np.sqrt(1/3), np.sqrt(1/3)
psi = np.array([alpha, beta, gamma, 0.0], dtype=complex)

input_state = psi

### Première option: utiliser fock_prob
Strawberryfields permet de calculer directement les probabilités d'un circuit en utilisant la méthode fock_prob() sur un état (ici res.state où res est le résultat du circuit).

In [7]:
prog = sf.Program(3)
with prog.context as q:
    ops.Fock(1)             | q[1]
    ops.Ket(input_state)    | q[0]
    NS_1(q)


# prog.print()

# First we define the Strawberryfields engine
eng = sf.Engine("fock", backend_options={"cutoff_dim": cutoff})
# Then we run our circuit
res = eng.run(prog)
# Here we recover the state of the system
state = res.state
# We add the cumulated probabilities of every success state
proba = 0.0
for n0 in range(4):
    proba += state.fock_prob([n0, 1, 0])  # Success for (n1=1,n2=0)
print("Success probability of the NS gate:")
print("P(a1=1,a2=0) =", proba)


Success probability of the NS gate:
P(a1=1,a2=0) = 0.25000000121135985


### Deuxième option: boucle à la main
Pour simuler plus classiquement un nombre fixé d'itération du circuit, puis calculer la probabilité de réussite.

In [6]:
def one_shot():
    prog = sf.Program(3)
    with prog.context as q:
        ops.Fock(1)             | q[1]  # ancilla |1>
        ops.Ket(input_state)    | q[0]  # état d'entrée
        NS_1(q)
        ops.MeasureFock()       | (q[1], q[2])  # mesurer seulement q1, q2

    eng = sf.Engine("fock", backend_options={"cutoff_dim": cutoff})
    res = eng.run(prog, shots=1)
    return res.samples[0]  # (n1, n2)

# Exemple: dix-milles tirs, WARNING: semble exponentiel en N
N = 100
samples = np.array([one_shot() for _ in range(N)])   # shape (N, 2)
print("Aperçu:", samples[:10])
success = np.mean((samples[:,0] == 1) & (samples[:,1] == 0))
print("P(succès post-sélection q1=1,q2=0) =", success)


Aperçu: [[1 0]
 [0 1]
 [2 0]
 [0 0]
 [3 0]
 [2 0]
 [0 1]
 [0 0]
 [0 2]
 [0 0]]
P(succès post-sélection q1=1,q2=0) = 0.25


## Figure 2: $C-Z$ gate (proba 1/16)

In [8]:
alpha, beta = np.sqrt(0.5), np.sqrt(0.5)

stateQ = np.zeros(cutoff**2, dtype=complex)

# |0,1>  → n0=0, n1=1
stateQ[0 * cutoff + 1] = alpha

# |1,0>  → n0=1, n1=0
stateQ[1 * cutoff + 0] = beta

print(stateQ.shape)  # (cutoff**2,)


# Produit tensoriel 8-modes: (0,1,2,3,4,5,6,7)
input_state = tensor_ket([stateQ, stateQ, ket1, ket0, ket1, ket0])

print(input_state)

(16,)
[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]


### fock_prob

In [9]:
prog = sf.Program(8)
with prog.context as q:
    # 1) Préparer d’un seul coup TOUS les modes
    ops.Ket(input_state) | q

    # 2) Circuit: BS +45° sur (0,2) ; deux NS ; BS -45° sur (0,2)
    # BS pour intrication
    ops.BSgate(angle45, 0.0) | (q[0], q[2])

    # NS sur 0 avec ancillas (4=|1>,5=|0>)
    NS_1([q[0],q[4],q[5]])

    # NS sur 2 avec ancillas (6=|1>,7=|0>)
    NS_1([q[2],q[6],q[7]])

    # Dernier BS
    ops.BSgate(-angle45, 0.0) | (q[0], q[2])

eng = sf.Engine("fock", backend_options={"cutoff_dim": 4})
res = eng.run(prog)  # pas de mesure: on lit les probas exactes ensuite

state = res.state

# ---- Proba de succès (n4,n5,n6,n7)=(1,0,1,0) ----
probs = state.all_fock_probs()  # shape 8x4
p_success = probs[:, :, :, :, 1, 0, 1, 0].sum()
print("Success probability of the Figure 2 (C-Z)")
print("P(q4=1,q5=0,q6=1,q7=0) =", float(p_success))


Success probability of the Figure 2 (C-Z)
P(q4=1,q5=0,q6=1,q7=0) = 0.062499999268879516


Différence entre 5% et 1/16 due à l'intrication de 0 et 2, rendant les portes NS dépendantes.

## Téléportation

### fock_prob

In [ ]:
input_state = psi

prog = sf.Program(3)
with prog.context as q:
    ops.Fock(1)             | q[2]
    ops.Fock(0)             | q[1]
    ops.Ket(input_state)    | q[0]
    
    ops.BSgate(angle45, 0.0)    | (q[1], q[2])
    ops.BSgate(-angle45, 0.0)   | (q[0], q[1])


eng = sf.Engine("fock", backend_options={"cutoff_dim": cutoff})
res = eng.run(prog)
state = res.state

probs = state.all_fock_probs()  # shape: (cutoff,)*12

p_success = 0.0
for n0 in range(cutoff):
    for n1 in range(cutoff):
        if (n0 + n1) % 2 == 1:  # parité impaire
            p_success += probs[n0, n1, :].sum()

print("Success probability of teleportation")
print("Probabilité de succès (parités impaires) =", float(p_success))


Success probability of teleportation
Probabilité de succès (parités impaires) = 0.5000000000000002


## Figure 3: $C-Z$ gate (proba 1/4)

### fock_prob

In [ ]:
# 4 mêmes entrées
psi0 = psi1 = psi2 = psi3 = psi

# Produit tensoriel 12-modes:
input_state = tensor_ket([psi0, psi1, psi2, psi3, ket0, ket1, ket0, ket1, ket1, ket0, ket1, ket0])

prog = sf.Program(12)
with prog.context as q:
    # 1) Préparation des modes
    ops.Ket(input_state) | (q[0], q[1], q[2], q[3], q[4], q[5], q[6], q[7], q[8], q[9], q[10], q[11])

    # 2) Circuit: 
    # BS pour intrication de 4,5,6,7
    ops.BSgate(-angle45, 0.0) | (q[4], q[5])
    ops.BSgate(angle45, 0.0) | (q[6], q[7])
    ops.BSgate(angle45, 0.0) | (q[5], q[7])

    # NS sur 5 avec ancillas (8=|1>,9=|0>)
    ops.Rgate(phi4)          | q[5]
    ops.BSgate(theta1,0.0)   | (q[8], q[9])   # (a1=|1>, a2=|0>)
    ops.BSgate(theta2,0.0)   | (q[5], q[8])   # (target, a1)
    ops.BSgate(theta3,0.0)   | (q[8], q[9])

    # NS sur 7 avec ancillas (10=|1>,11=|0>)
    ops.Rgate(phi4)          | q[7]
    ops.BSgate(theta1,0.0)   | (q[10], q[11])
    ops.BSgate(theta2,0.0)   | (q[7], q[10])
    ops.BSgate(theta3,0.0)   | (q[10], q[11])

    # Dernier BS
    ops.BSgate(-angle45, 0.0) | (q[5], q[7])
    ops.BSgate(angle45, 0.0) | (q[0], q[4])
    ops.BSgate(-angle45, 0.0) | (q[2], q[6])


eng = sf.Engine("fock", backend_options={"cutoff_dim": cutoff})
res = eng.run(prog)
state = res.state

probs = state.all_fock_probs()  # shape: 12x4

p_success = 0.0
for n0 in range(cutoff):
    for n4 in range(cutoff):
        if (n0 + n4) % 2 == 1:  # parité impaire (0,4)
            for n2 in range(cutoff):
                for n6 in range(cutoff):
                    if (n2 + n6) % 2 == 1:  # parité impaire (2,6)
                        # somme sur tous les autres modes : 1,3,5,7,8,9,10,11
                        p_success += probs[
                            n0, :, n2, :, n4, :, n6, :, :, :, :
                        ].sum()

print("Success probability of the Figure 3 (C-Z)")
print("Probabilité de succès (parités impaires) =", float(p_success))


Success probability of the Figure 3 (C-Z)
Probabilité de succès (parités impaires) = 0.25000000000000033
